In [ ]:
# env.py
import airsim
import numpy as np
import cv2

class DroneEnv:
    def __init__(self):
        self.client = airsim.MultirotorClient()
        self.client.confirmConnection()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        self.image_shape = (84, 84, 3)  # Resize shape

    def reset(self):
        self.client.reset()
        self.client.enableApiControl(True)
        self.client.armDisarm(True)
        self.client.takeoffAsync().join()
        return self.get_state()

    def get_state(self):
        responses = self.client.simGetImages([
            airsim.ImageRequest("0", airsim.ImageType.Scene, False, False)
        ])
        img1d = np.frombuffer(responses[0].image_data_uint8, dtype=np.uint8)
        img = img1d.reshape(responses[0].height, responses[0].width, 3)
        img = cv2.resize(img, (self.image_shape[1], self.image_shape[0])) / 255.0
        return img

    def step(self, action):
        self.client.moveByAngleRatesThrottleAsync(
            pitch_rate=action[0],
            roll_rate=action[1],
            yaw_rate=action[2],
            throttle=action[3],
            duration=0.05
        ).join()

        state = self.get_state()
        collision = self.client.simGetCollisionInfo().has_collided
        reward = -100 if collision else 1  # Basic survival reward
        done = collision
        return state, reward, done
